In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torch.autograd import Variable

In [39]:
BATCH_SIZE = 30
SEQ_LENGTH = 273
EMBEDDING_DIM = 60
class ActionNet(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, label_size, batch_size):
        super(ActionNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.batch_size = batch_size
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)
        self.hidden2label = nn.Linear(SEQ_LENGTH*hidden_dim, label_size)
        self.hidden = self.init_hidden()
        
    def init_hidden(self):
        # the first is the hidden h
        # the second is the cell  c
        return (autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)).cuda(),
                autograd.Variable(torch.zeros(1, self.batch_size, self.hidden_dim)).cuda())

    def forward(self, x):
        #print(x.shape)
        lstm_out, self.hidden = self.lstm(x, self.hidden)
        y  = self.hidden2label(lstm_out.view(BATCH_SIZE,-1))
        log_probs = F.softmax(y, dim=1)
        return y

net = torch.load("action_net.pt")

In [40]:
def pred(x):
    out = net(x)
    weight, action = torch.max(out, dim=1)
    return action.data.cpu().numpy()

In [41]:
test_data = np.load('test_np.npy')

In [42]:
test_data.shape

(1, 273, 20, 3)

In [43]:
test_tensor = torch.Tensor(test_data)
test_tensor = test_tensor.repeat(30,1,1,1)
test_tensor = test_tensor.view(-1,273,30,60)
test_var = Variable(test_tensor)

In [44]:
test_var.shape

torch.Size([1, 273, 30, 60])

In [45]:
#test_batch_x = x[i*BATCH_SIZE:(i+1)*BATCH_SIZE].view(-1,SEQ_LENGTH, BATCH_SIZE, EMBEDDING_DIM)
pred(test_var[0].cuda())

/usr/local/lib/python3.5/dist-packages/ipykernel-4.8.1-py3.5.egg/ipykernel_launcher.py:22: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greately increasing memory usage. To compact weights again call flatten_parameters().


array([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
       4, 4, 4, 4, 4, 4, 4, 4])